In [12]:
import pandas as pd
from bs4 import BeautifulSoup  
import re
import nltk
from nltk.corpus import stopwords # Import the stop word list

train = pd.read_csv("labeledTrainData.tsv",
            header =0,
            delimiter = "\t", 
            quoting=3
           )



In [36]:
def review_to_words( raw_review):
    #function to clean a single review
    example1 = BeautifulSoup(raw_review).get_text() #Review HTML XML
    letters_only = re.sub("[^a-zA-Z]", #The patter to search for, Everything except the following.
                     " ", # the pattern to replace it with
                     example1 #The text to search for
                     )
    
    words = letters_only.lower().split()
    #remove stop words from "words"
    stops = set(stopwords.words("english"))
    words = [w
            for w in words
            if not w in stops]
    return (" ".join(words))

In [71]:
clean_train_reviews = [review_to_words(w) for w in train["review"]]

In [64]:
from sklearn.feature_extraction.text import CountVectorizer


In [72]:
vectorizer = CountVectorizer(analyzer = "word",
                            tokenizer = None,
                            preprocessor = None,
                            stop_words = None,
                            max_features = 5000)


In [74]:
train_data_features = vectorizer.fit_transform(clean_train_reviews)

In [75]:
train_data_features = train_data_features.toarray()

In [92]:
vocab = vectorizer.get_feature_names()

In [94]:
import numpy as np

dist = np.sum(train_data_features, axis =0)

In [95]:
dist

array([187, 125, 108, ..., 740, 518, 147])

In [98]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 100)

In [99]:
forest = forest.fit (train_data_features, train["sentiment"])

In [100]:
test = pd.read_csv("testData.tsv",
                  header =0,
                  delimiter ="\t",
                  quoting =3 )

print test.shape

(25000, 2)


In [109]:
num_reviews = len(test["review"])
clean_test_reviews  = []

In [110]:
for i in xrange(0,num_reviews):
    if ((i+1) %1000 == 0):
        print "Review %d  of %d\n" %(i+1,num_reviews)
    clean_review = review_to_words( test["review"][i])
    clean_test_reviews.append(clean_review)
    
    
test_data_features = vectorizer.transform(clean_test_reviews).toarray()

result = forest.predict(test_data_features)


Review 1000  of 25000

Review 2000  of 25000

Review 3000  of 25000

Review 4000  of 25000

Review 5000  of 25000

Review 6000  of 25000

Review 7000  of 25000

Review 8000  of 25000

Review 9000  of 25000

Review 10000  of 25000

Review 11000  of 25000

Review 12000  of 25000

Review 13000  of 25000

Review 14000  of 25000

Review 15000  of 25000

Review 16000  of 25000

Review 17000  of 25000

Review 18000  of 25000

Review 19000  of 25000

Review 20000  of 25000

Review 21000  of 25000

Review 22000  of 25000

Review 23000  of 25000

Review 24000  of 25000

Review 25000  of 25000



In [111]:
    output = pd.DataFrame (data={"id":test["id"],"sentiment":result})

    output.to_csv("Bag_of_words_model.csv",index=False, quoting=3)
